In [10]:
# Code to Perform Block Matching

import numpy as np
import cv2
import os
import math    

In [11]:
def YCrCb2BGR(image):
    """
    Converts numpy image into from YCrCb to BGR color space
    """
    return cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

def BGR2YCrCb(image):
    """
    Converts numpy image into from BGR to YCrCb color space
    """
    return cv2.cvtColor(image, cv2.COLOR_YCrCb2BGR)

def segmentImage(anchor, blockSize=16):
    """
    Determines how many macroblocks an image is composed of
    :param anchor: I-Frame
    :param blockSize: Size of macroblocks in pixels
    :return: number of rows and columns of macroblocks within
    """
    h, w = anchor.shape
    hSegments = int(h / blockSize)
    wSegments = int(w / blockSize)
    totBlocks = int(hSegments * wSegments)


    print(f"Height: {h}, Width: {w}")
    print(f"Segments: Height: {hSegments}, Width: {wSegments}")
    print(f"Total Blocks: {totBlocks}")

    return hSegments, wSegments

def getCenter(x, y, blockSize):
    
    ## Determines center of a block with x, y as top left corner coordinates and blockSize as blockSize
    ## return: x, y coordinates of center of a block
    
    return (int(x + blockSize/2), int(y + blockSize/2))

def getAnchorSearchArea(x, y, anchor, blockSize, searchArea):
    """
    Returns image of anchor search area
    :param x, y: top left coordinate of macroblock in Current Frame
    :param anchor: I-Frame
    :param blockSize: size of block in pixels
    :param searchArea: size of search area in pixels
    :return: Image of anchor search area
    """
    h, w = anchor.shape
    cx, cy = getCenter(x, y, blockSize)

    sx = max(0, cx-int(blockSize/2)-searchArea) # ensure search area is in bounds
    sy = max(0, cy-int(blockSize/2)-searchArea) # and get top left corner of search area

    # slice anchor frame within bounds to produce anchor search area
    anchorSearch = anchor[sy:min(sy+searchArea*2+blockSize, h), sx:min(sx+searchArea*2+blockSize, w)]

    return anchorSearch

In [12]:
def getBlockZone(p, aSearch, tBlock, blockSize):
    """
    Retrieves the block searched in the anchor search area to be compared with the macroblock tBlock in the current frame
    :param p: x,y coordinates of macroblock center from current frame
    :param aSearch: anchor search area image
    :param tBlock: macroblock from current frame
    :param blockSize: size of macroblock in pixels
    :return: macroblock from anchor
    """
    px, py = p # coordinates of macroblock center
    px, py = px-int(blockSize/2), py-int(blockSize/2) # get top left corner of macroblock
    px, py = max(0,px), max(0,py) # ensure macroblock is within bounds

    aBlock = aSearch[py:py+blockSize, px:px+blockSize] # retrive macroblock from anchor search area


    # try:
    #     assert aBlock.shape == tBlock.shape # must be same shape

    # except Exception as e:
    #     print(e)
    #     print(f"ERROR - ABLOCK SHAPE: {aBlock.shape} != TBLOCK SHAPE: {tBlock.shape}")

    return aBlock

In [13]:
def getMSE(tBlock, aBlock):
    # Ensure that both blocks have the same shape
    if tBlock.shape != aBlock.shape:
        # Resize blocks to have the same shape
        min_rows = min(tBlock.shape[0], aBlock.shape[0])
        min_cols = min(tBlock.shape[1], aBlock.shape[1])
        tBlock_resized = cv2.resize(tBlock, (min_cols, min_rows))
        aBlock_resized = cv2.resize(aBlock, (min_cols, min_rows))
    else:
        tBlock_resized = tBlock
        aBlock_resized = aBlock

    return np.sum(np.square(np.subtract(tBlock_resized, aBlock_resized))) / (tBlock_resized.shape[0] * tBlock_resized.shape[1])


In [14]:
def getBestMatchLogarithmic(tBlock, aSearch, blockSize):
    """
    Implemented Logarithmic Search instead of 3 Step Search.
    :param tBlock: macroblock from the current frame
    :param aSearch: anchor search area
    :param blockSize: size of macroblock in pixels
    :return: macroblock from anchor search area with least MSE
    """
    ah, aw = aSearch.shape
    acy, acx = int(ah/2), int(aw/2)  # get center of anchor search area

    minMSE = float("+inf")
    minP = None

    max_radius = max(acy, acx)
    radius = max_radius

    while radius > 1:
        theta = 0
        while theta < 2 * math.pi:
            x_offset = int(radius * math.cos(theta))
            y_offset = int(radius * math.sin(theta))

            px, py = (acx + x_offset, acy + y_offset)

            px, py = px - int(blockSize / 2), py - int(blockSize / 2)  # get top left corner of point
            px, py = max(0, px), max(0, py)  # ensure point is within bounds

            aBlock = getBlockZone((px, py), aSearch, tBlock, blockSize)  # get anchor macroblock
            MSE = getMSE(tBlock, aBlock)  # determine MSE

            if MSE < minMSE:  # store point with minimum MSE
                minMSE = MSE
                minP = (px + int(blockSize / 2), py + int(blockSize / 2))  # center of anchor block with minimum MSE

            theta += math.pi / 8  # 45 degrees increment

        radius = int(radius / 2)

    px, py = minP
    px, py = px - int(blockSize / 2), py - int(blockSize / 2)  # get top left corner of minP
    px, py = max(0, px), max(0, py)  # ensure minP is within bounds
    matchBlock = aSearch[py:py + blockSize, px:px + blockSize]  # retrieve best macroblock from anchor search area

    return matchBlock


In [15]:
def blockSearchBody(anchor, target, blockSize, searchArea=7):
    """
    Facilitates the creation of a predicted frame based on the anchor and target frame
    :param anchor: I-Frame
    :param target: Current Frame to create a P-Frame from
    :param blockSize: size of macroBlock in pixels
    :param searchArea: size of searchArea extended from blockSize
    :return: predicted frame
    """
    h, w = anchor.shape
    hSegments, wSegments = segmentImage(anchor, blockSize)

    predicted = np.ones((h, w)) * 255
    bcount = 0
    for y in range(0, int(hSegments * blockSize), blockSize):
        for x in range(0, int(wSegments * blockSize), blockSize):
            bcount += 1
            targetBlock = target[y:y + blockSize, x:x + blockSize]  # get current macroblock

            anchorSearchArea = getAnchorSearchArea(x, y, anchor, blockSize, searchArea)  # get anchor search area

            anchorBlock = getBestMatchLogarithmic(targetBlock, anchorSearchArea, blockSize)  # get best anchor macroblock

            # Ensure the shapes match by resizing the anchorBlock to match the targetBlock size
            anchorBlock = cv2.resize(anchorBlock, (blockSize, blockSize))

            predicted[y:y + blockSize, x:x + blockSize] = anchorBlock  # add anchor block to predicted frame

    assert bcount == int(hSegments * wSegments)  # check all macroblocks are accounted for

    return predicted


In [16]:
def getResidual(target, predicted):
    """Create residual frame from target frame - predicted frame"""
    return np.subtract(target, predicted)

def getReconstructTarget(residual, predicted):
    """Reconstruct target frame from residual frame plus predicted frame"""
    return np.add(residual, predicted)

def getResidualMetric(residualFrame):
    """Calculate residual metric from average of sum of absolute residual values in residual frame"""
    return np.sum(np.abs(residualFrame))/(residualFrame.shape[0]*residualFrame.shape[1])


In [17]:
def preprocess(anchor, target, blockSize):

    if isinstance(anchor, str) and isinstance(target, str):
        anchorFrame = BGR2YCrCb(cv2.imread(anchor))[:, :, 0] # get luma component
        targetFrame = BGR2YCrCb(cv2.imread(target))[:, :, 0] # get luma component

    elif isinstance(anchor, np.ndarray) and isinstance(target, np.ndarray):
        anchorFrame = BGR2YCrCb(anchor)[:, :, 0] # get luma component
        targetFrame = BGR2YCrCb(target)[:, :, 0] # get luma component

    else:
        raise ValueError

    #resize frame to fit segmentation
    hSegments, wSegments = segmentImage(anchorFrame, blockSize)
    anchorFrame = cv2.resize(anchorFrame, (int(wSegments*blockSize), int(hSegments*blockSize)))
    targetFrame = cv2.resize(targetFrame, (int(wSegments*blockSize), int(hSegments*blockSize)))

    #if debug:
        #print(f"A SIZE: {anchorFrame.shape}")
        #print(f"T SIZE: {targetFrame.shape}")


    return (anchorFrame, targetFrame)

In [18]:
def main(anchorFrame, targetFrame, outfile="OUTPUT", blockSize = 16):
    """
    Calculate residual frame and metric along with other artifacts
    :param anchor: file path of I-Frame or I-Frame
    :param target: file path of Current Frame or Current Frame
    :return: residual metric
    """
    anchorFrame, targetFrame = preprocess(anchorFrame, targetFrame, blockSize) #processes frame or filepath to frame

    predictedFrame = blockSearchBody(anchorFrame, targetFrame, blockSize)
    residualFrame = getResidual(targetFrame, predictedFrame)
    naiveResidualFrame = getResidual(anchorFrame, targetFrame)
    reconstructTargetFrame = getReconstructTarget(residualFrame, predictedFrame)

    residualMetric = getResidualMetric(residualFrame)
    naiveResidualMetric = getResidualMetric(naiveResidualFrame)

    rmText = f"Residual Metric: {residualMetric:.2f}"
    nrmText = f"Naive Residual Metric: {naiveResidualMetric:.2f}"

    isdir = os.path.isdir(outfile)
    if not isdir:
        os.mkdir(outfile)

    
    cv2.imwrite("OUTPUT/targetFrame.png", targetFrame)
    cv2.imwrite("OUTPUT/predictedFrame.png", predictedFrame)
    cv2.imwrite("OUTPUT/residualFrame.png", residualFrame)
    cv2.imwrite("OUTPUT/reconstructTargetFrame.png", reconstructTargetFrame)
    cv2.imwrite("OUTPUT/naiveResidualFrame.png", naiveResidualFrame)
    resultsFile = open("OUTPUT/results.txt", "w"); resultsFile.write(f"{rmText}\n{nrmText}\n"); resultsFile.close()

    print(rmText)
    print(nrmText)

    return residualMetric, residualFrame

if __name__ == "__main__":
    anchorPath = "C://Users//Rafik//Desktop//Master Miv//CM\Projects//Test//Frame04.jpg"
    targetPath = "C://Users//Rafik//Desktop//Master Miv//CM\Projects//Test//Frame05.jpg"
    # anchorPath = "C://Users//Rafik//Desktop//Master Miv//CM\Projects//Test//Frame02.png"
    # targetPath = "C://Users//Rafik//Desktop//Master Miv//CM\Projects//Test//Frame02.png"
    main(anchorPath, targetPath)

Height: 3472, Width: 4640
Segments: Height: 217, Width: 290
Total Blocks: 62930
Height: 3472, Width: 4640
Segments: Height: 217, Width: 290
Total Blocks: 62930
Residual Metric: 34.84
Naive Residual Metric: 82.45
